In [1]:
""" A simple demo of GP Breguet range formulations that do and do not work"""
from gpkit import Variable, Model
from gpkit.tools import te_exp_minus1
import numpy as np
from gpkit.constraints.tight import TightConstraintSet as TCS

In [12]:
class BreguetRange(Model):
    """
    A Breguet range formulation fully compatbile with GP
    """
    def __init__(self, **kwargs):
        #Variable definitions
        TSFC = Variable('TSFC', '1/hr', 'Thrust Specific Fuel Consumtion')
        z_bre = Variable('z_bre', '-', 'Breguet Range Parameter')
        D = Variable('D', 'N', 'Drag')
        rho = Variable('rho', 'kg/m^3', 'Air Density')
        V = Variable('V', 'kts', 'Cruise Airspeed')
        Cd0 = Variable('Cd0', '-', 'Profile Drag Coefficient')
        K = Variable('K', '-', 'Parametric Drag Model Parameter')
        W_fuel = Variable('W_fuel', 'N', 'Segment Fuel Weight')
        W_end = Variable('W_end', 'N', 'Segment End Weight')
        W_avg = Variable('W_avg', 'N', 'Average Segment Weight')
        W_start = Variable('W_start', 'N', 'Segment Start Weight')
        Range = Variable('Range', 'mi', 'Required Segment Range')
        t = Variable('t', 'hr', 'Segment Flight Time')
        Cl = Variable('Cl', '-', 'Segment Lift Coefficient')
        S = Variable('S', 'm^2', 'Wing Planform Area')
        LD = Variable('L/D', '-', 'Lift to Drag Ratio')
        W_limit = Variable('W_limit', 'N', 'Non-Physical Weight Limit Needed for Convergence')
        
        
        constraints = []
        
        #write out all required constraints
        constraints.extend([
                #Breguet Range parameter constraints
                TCS([W_fuel/W_end >= te_exp_minus1(z_bre,3)]),
                TCS([z_bre >= TSFC * t * D/ W_end]),
                
                #constraint on the lift coefficient, assumes steady level flight
                W_avg == .5 * Cl * S * rho * V**2,
                
                #drag constraint
                TCS([D >= (.5*S*rho*V**2)*(Cd0 + K * Cl**2)]),
                
                #constrain the starting weight such that the aircraft is only losing weight due to fuel burn
                TCS([W_start >= W_end + W_fuel]),
                
                #average weight is the geometric mean of the start and end weights
                W_avg == (W_start * W_end)**.5,
                
                #constraint on the segment time
                t * V == Range,
                
                #non-physical weight limit for demonstration purposes, relevant only in the second Breguet formulation
                W_start <= W_limit,
            ])
        
        #substitute values in for variables that are constants
        substitutions = {
            'S': 125,                 #approx a B737 wing area
            'Range': 1000,            #1,000 mile required range
            'W_end': 40000*9.81,      #approx empty weight of B737 in N
            'V': 420,                 #set the cruise speed
            'rho': .31,               #air density at 12,000m (~40,000')
            'Cd0': .02,               #setting profile drag coefficient
            'K': .015,                #setting parametric drag model coefficient
            'TSFC': 0.5,              #setting segment TSFC
            'W_limit': 100000*9.81,
        }
        
        #build the model
        Model.__init__(self, W_fuel, constraints, substitutions)

In [11]:
#create the model
Breguet = BreguetRange()
#solve the model, noting the lack of Tight Constraint Set Warnings
sol = Breguet.solve(solver = "cvxopt")
#print the solution
print sol.table()

Using solver 'cvxopt'
Solving for 7 variables.
Solving took 0.00871 seconds.

Cost
----
 2.21e+04 [N] 

Free Variables
--------------
     Cl : 0.4459          Segment Lift Coefficient
      D : 2.079e+04  [N]  Drag                    
  W_avg : 4.033e+05  [N]  Average Segment Weight  
 W_fuel : 2.21e+04   [N]  Segment Fuel Weight     
W_start : 4.145e+05  [N]  Segment Start Weight    
      t : 2.069      [hr] Segment Flight Time     
  z_bre : 0.0548          Breguet Range Parameter 

Constants
---------
    Cd0 : 0.02                 Profile Drag Coefficient                        
      K : 0.015                Parametric Drag Model Parameter                 
  Range : 1000       [mi]      Required Segment Range                          
      S : 125        [m**2]    Wing Planform Area                              
   TSFC : 0.5        [1/hr]    Thrust Specific Fuel Consumtion                 
      V : 420        [kt]      Cruise Airspeed                                 
  W_end 

In [103]:
#Breguet[0] = TCS([Breguet["W_fuel"]/Breguet["W_end"] >= te_exp_minus1(Breguet["z_bre"],3)])
#print Breguet[1]
#Breguet[1] = TCS([Breguet["z_bre"]/Breguet["W_end"] >= te_exp_minus1(Breguet["z_bre"],3)])

In [104]:
class Non_Physical_Breguet_Range(Model):
    """
    A Breguet range formulation which GP solves, but yields a non-phycial result
    """
    def __init__(self, **kwargs):
        #Variable definitions
        TSFC = Variable('TSFC', '1/hr', 'Thrust Specific Fuel Consumtion')
        z_bre = Variable('z_bre', '-', 'Breguet Range Parameter')
        D = Variable('D', 'N', 'Drag')
        rho = Variable('rho', 'kg/m^3', 'Air Density')
        V = Variable('V', 'kts', 'Cruise Airspeed')
        Cd0 = Variable('Cd0', '-', 'Profile Drag Coefficient')
        K = Variable('K', '-', 'Parametric Drag Model Parameter')
        W_fuel = Variable('W_fuel', 'N', 'Segment Fuel Weight')
        W_end = Variable('W_end', 'N', 'Segment End Weight')
        W_avg = Variable('W_avg', 'N', 'Average Segment Weight')
        W_start = Variable('W_start', 'N', 'Segment Start Weight')
        Range = Variable('Range', 'mi', 'Required Segment Range')
        t = Variable('t', 'hr', 'Segment Flight Time')
        Cl = Variable('Cl', '-', 'Segment Lift Coefficient')
        S = Variable('S', 'm^2', 'Wing Planform Area')
        LD = Variable('L/D', '-', 'Lift to Drag Ratio')
        W_limit = Variable('W_limit', 'N', 'Non-Physical Weight Limit Needed for Convergence')
        
        constraints = []
        
        #write out all required constraints
        constraints.extend([
                #drag constraint
                TCS([D >= (.5*S*rho*V**2)*(Cd0 + K * Cl**2)]),
                
                #Breguet Range parameter constraints
                TCS([W_fuel/W_end >= te_exp_minus1(z_bre,3)]),
                TCS([Range <= z_bre * LD * V / TSFC]),
                
                #compute L/D
                LD == W_avg/D,
                
                #constrain the starting weight such that the aircraft is only losing weight due to fuel burn
                TCS([W_start >= W_end + W_fuel]),
                
                #average weight is the geometric mean of the start and end weights
                W_avg == (W_start * W_end)**.5,
                
                #constraint on the lift coefficient, assumes steady level flight
                W_avg == .5 * Cl * S * rho * V**2,
                
                #constraint on the segment time
                t * V == Range,
                
                #non-physical weight limit for demonstration purposes
                TCS([W_start <= W_limit]),
            ])
        
        #substitute values in for variables that are constants
        substitutions = {
            'S': 125,                 #approx a B737 wing area
            'Range': 1000,            #1,000 mile required range
            'W_end': 40000*9.81,      #approx empty weight of B737 in N
            'V': 420,                 #set the cruise speed
            'rho': .31,               #air density at 12,000m (~40,000')
            'Cd0': .02,               #setting profile drag coefficient
            'K': .015,                #setting parametric drag model coefficient
            'TSFC': 0.5,              #setting segment TSFC
            'W_limit': 100000*9.81,
        }
        
        #build the model
        Model.__init__(self, W_fuel, constraints, substitutions)

In [105]:
#create the model
NonPhysicalBreguet = Non_Physical_Breguet_Range()
#solve the model, noting the lack of Tight Constraint Set Warnings
NonPhysicalSol = NonPhysicalBreguet.solve()
#print the solution
print NonPhysicalSol.table()

Using solver 'cvxopt'
Solving for 8 variables.
Solving took 0.00827 seconds.


Cost
----
 1.634e+04 [N] 

Free Variables
--------------
     Cl : 0.6859          Segment Lift Coefficient
      D : 2.447e+04  [N]  Drag                    
    L/D : 25.35           Lift to Drag Ratio      
  W_avg : 6.204e+05  [N]  Average Segment Weight  
 W_fuel : 1.634e+04  [N]  Segment Fuel Weight     
W_start : 9.81e+05   [N]  Segment Start Weight    
      t : 2.069      [hr] Segment Flight Time     
  z_bre : 0.04081         Breguet Range Parameter 

Constants
---------
    Cd0 : 0.02                 Profile Drag Coefficient                        
      K : 0.015                Parametric Drag Model Parameter                 
  Range : 1000       [mi]      Required Segment Range                          
      S : 125        [m**2]    Wing Planform Area                              
   TSFC : 0.5        [1/hr]    Thrust Specific Fuel Consumtion                 
      V : 420        [kt]      Crui